In [ ]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 9
### Team Member Names: Germain, Padtmesh, Jin
### Team Strategy Chosen: Safe

In [ ]:
def filterByDenom(ticker):
    # Check if a stock is a "US listed stock"
    return ticker.info['currency'] != 'USD'

def filterByVol(hist):
    # Check the average monthly volume
    return  hist['Volume'].mean() < 200000

def filterByTDays(df_hist):
    df = df_hist.copy()
    # Iterate across time range with months January to October
    for month in range(1, 11):
        # Get numerical month value of each index
        idx = pd.to_datetime(df.index).month
        # Get subset of df_hist of a specific month
        df_month = df.loc[idx == month]
        # Filter by number of entries/trading days for a specific month 
        if (len(df_month) < 20):
            df.drop(df[idx == month].index, inplace=True)
    return df



In [ ]:
def getHist(tickers):
    # Set defaults for timeframe
    start_date = '2022-01-01'
    end_date = '2022-11-01' # adjustment for non-inclusive param end

    # Create dataframe to hold historic data from tickers
    df_sv = pd.DataFrame()

    # Get the closing price of each ticker 
    for tick in tickers['Tickers']:
        # Call API for ticker data
        temp = yf.Ticker(tick)

        # Get historical data
        #   Note: the method history() prints out the delisted message
        temp_hist = temp.history(start=start_date, end=end_date, interval='1d')

        # Filter by denom and volume of non-delisted tickers
        if (not temp_hist.empty) and (not (filterByDenom(temp) or filterByVol(temp_hist))):
            # Get Closing Price as a DataFrame
            df_sv[tick] = pd.DataFrame(temp_hist['Close'])

    # Remove indeterminate or faulty values
    df_sv.dropna(axis=1,how='all',inplace = True)
    df_sv.dropna(inplace = True)

    # Filter by number of trading days
    df_sv = filterByTDays(df_sv)
    return df_sv

In [ ]:
def sorHistPct(df_pct):
    # Sort arrays by std
    return df_pct.reindex(df_pct.std().sort_values().index, axis=1)

In [ ]:
def getReturns(pct, weights):
    # Calculate annuzalized returns
    return (np.sum(pct.mean() * weights)) * 253

def getStd(pct, weights):
    # Calculate annuzalized std
    return np.sqrt(np.transpose(weights) @ (pct.cov() * 253) @ weights)

def randSimulation(pct):
    # Number of random portolfios to generate
    # Possible consideration is n = 10**5 since it provides a more accurate approximation
        # Note: generation 10**x portfolios where x>5 produces negligible results
        #   and the benefits are largely outweighed by the runtime
    n = 10**4 
    
    # Get the number of tickers
    ticker_count = len(pct.columns)

    # Lists to store portfolio data
    list_return = []
    list_std = []
    list_weight = []

    # Generate n random portfolios
    for p in range(n):
        # Get random values between 0 and 1
        #   Note: we use the uniform generation method as we want the samples to be dispersed evenly 
        weights = np.random.uniform(size= ticker_count)

        # Calculating and establishing lower bound for random portfolio weights
        #   Note: in establishing this lower bound we no longer need to consider the upper bound as it 
        #       would be impossible for any one ticker to have more than 25% allocation with the given lower bound
        low = 0.02
        temp = weights
        temp = (temp/temp.sum()*(1-low*ticker_count))
        weights = temp+low

        # Store randomly generated portfolio data in lists
        list_weight.append(weights)
        list_return.append(getReturns(pct,weights))
        list_std.append(getStd(pct,weights))

    # Convert list to np arrays
    arr_return = np.array(list_return)
    arr_std = np.array(list_std)
    arr_weight = np.array(list_weight)

    return arr_return, arr_std, arr_weight



def graphSimulation(returns, stds, weights):
    # Get information for max sharpe ratio portfolio
    maxs_index = np.argmax(returns/stds)
    maxs_std = stds[maxs_index] 
    maxs_return = returns[maxs_index] 
    ms_weight = weights[maxs_index]
    
    # Get information for min volatility portfolio
    minv_index = np.argmin(stds)
    minv_std = stds[minv_index]
    minv_return = returns[minv_index]
    minv_weight = weights[minv_index]

    print('Min Volatility Portfolio Weights:')
    print(minv_weight)

    print('Min Volatility:')
    print(minv_std)

    # Adjust figure size
    plt.figure(figsize=(20, 10))

    # Plot generated portfolios
    plt.scatter(stds,returns, c=(returns/stds), cmap='winter', s=12, alpha=0.4)
    plt.colorbar(label='Sharpe Ratio', shrink=0.7, cmap='winter')

    # Plot max sharpe ratio portoflio
    plt.scatter(maxs_std,maxs_return,c='plum' , s=120,marker='D', label='Maximum Sharpe Ratio Portfolio')
    # Plot min volatility portfolio
    plt.scatter(minv_std,minv_return,c='palevioletred', s=120, marker='D',label='Minimum Volatility Portfolio')

    # Labels and elements 
    plt.legend()
    plt.title(' Volatility-Returns of Generated Portfolios')
    plt.xlabel('Annualized Portfolio Volatility')
    plt.ylabel('Annualized Portfolio Returns')
    

In [ ]:
# def createPortFinal():


In [ ]:
# def createStocksFinal():


In [ ]:
# Read in csv file containing tickers
tickers = pd.read_csv('Tickers.csv', names = ['Tickers'])
display(tickers)

In [ ]:
# Get Historic Data for each ticker
hist = getHist(tickers)
pd.set_option('display.max_rows', None)

In [ ]:
# Construct dataframe of percentage change of each stock
hist_pct = hist.pct_change()

# Sort hist_pct based on std
hist_pct = sorHistPct(hist_pct)

# Create new dataframe inplace from the first 25 tickers (25 lowest std)
hist_pct = hist_pct.iloc[:,:25]

In [ ]:
# Get data from randomly generated portfolios
gen_return, gen_std, gen_weights = randSimulation(hist_pct)

In [ ]:
# Graph randomly generated portfolios
graphSimulation(gen_return, gen_std, gen_weights)

In [ ]:
# Create Portfolio Final and Stocks Final here

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.